### Install packages

In [88]:
!pip install selenium

In [89]:
!pip install html5lib

In [ ]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

import pandas as pd

### Extract data from the website from 2020 to 2025

In [ ]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

service = Service(executable_path='C:\Program Files\Google\Chrome\Application\chromedriver.exe')
driver = webdriver.Chrome(service=service)

link_1 = 'https://www.housingauthority.gov.hk/en/home-ownership/hos-secondary-market/transaction-records/transaction-records-search-detail-by-month.html?catId=1&para0='
link_2 = '&para1='

year = list(range(2020, 2026)) # from 2023 to 2025
month = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'] # from 1 to 12, add preceding 0 for going to the website

overall_transaction_df = pd.DataFrame()

for y in year:
    for m in month:
        current_date = datetime.date(y, int(m), 1)
        end_date = datetime.date(2025, 3, 1)
        
        if current_date > end_date:
            break

        print(y, m)
        link = link_1 + str(y) + link_2 + str(m)  
        driver.get(link)
        print(driver.title)


# Get the title of each table
        table_names = []

        table_title = driver.find_elements(By.CLASS_NAME,'main_section')
        for t in table_title:
            row = t.find_elements(By.CLASS_NAME,'transaction-records-district-title')
            for r in row:
                table_names.append(r.text)


# Get the header of each table
        headers_list = list()

        table = driver.find_elements(By.CLASS_NAME,'rwd-table')
        for t in table: # t is the table level
            rows = t.find_elements(By.TAG_NAME,'tr') # tr is row level
            for row in rows:
                cols = row.find_elements(By.TAG_NAME,'th') # td is each column field's data
                for col in cols:
                    headers_list.append(col.text)

        headers_list = list(dict.fromkeys(headers_list))
        headers_list = [item for item in headers_list if item.strip()] # remove empty string from the list

        if overall_transaction_df.empty:
            overall_transaction_df = pd.DataFrame(columns=headers_list)
            overall_transaction_df['District'] = None
            overall_transaction_df['Year'] = None
            overall_transaction_df['Month'] = None


        data_row_list = []
        row_data = []
        index = 0

        table = driver.find_elements(By.CLASS_NAME,'rwd-table')
        for i, t in enumerate(table): # t is the table level
            rows = t.find_elements(By.TAG_NAME,'tr') # tr is row level
            current_district = table_names[i]

            for row in rows:
                cols = row.find_elements(By.TAG_NAME,'td') # td is each column field's data
                for col in cols:
                    data_row_list.append(col.text)
                    index += 1

                    if index == len(headers_list): # append the static value after all the original data is appended
                        data_row_list.append(current_district)
                        data_row_list.append(y)
                        data_row_list.append(m)
                        row_data.append(data_row_list)
                        data_row_list = []
                        index = 0

        rows_df = pd.DataFrame(row_data, columns = overall_transaction_df.columns)
        overall_transaction_df = pd.concat([overall_transaction_df, rows_df], ignore_index = True)

        rows_df = pd.DataFrame(columns = overall_transaction_df.columns) # empty the rows_df for the next iteration


driver.quit()

<>:6: SyntaxWarning: invalid escape sequence '\P'
<>:6: SyntaxWarning: invalid escape sequence '\P'
C:\Users\david\AppData\Local\Temp\ipykernel_41792\3517073592.py:6: SyntaxWarning: invalid escape sequence '\P'
  service = Service(executable_path='C:\Program Files\Google\Chrome\Application\chromedriver.exe')


2020 01
Transaction Records | Hong Kong Housing Authority and Housing Department | Hong Kong Housing Authority and Housing Department
2020 02
Transaction Records | Hong Kong Housing Authority and Housing Department | Hong Kong Housing Authority and Housing Department
2020 03
Transaction Records | Hong Kong Housing Authority and Housing Department | Hong Kong Housing Authority and Housing Department
2020 04
Transaction Records | Hong Kong Housing Authority and Housing Department | Hong Kong Housing Authority and Housing Department
2020 05
Transaction Records | Hong Kong Housing Authority and Housing Department | Hong Kong Housing Authority and Housing Department
2020 06
Transaction Records | Hong Kong Housing Authority and Housing Department | Hong Kong Housing Authority and Housing Department
2020 07
Transaction Records | Hong Kong Housing Authority and Housing Department | Hong Kong Housing Authority and Housing Department
2020 08
Transaction Records | Hong Kong Housing Authority and 

In [ ]:
# 01/2020 to 03/2025 Home Ownership Scheme (HOS) secondary market transaction records
overall_transaction_df

### Data cleaning

In [ ]:
# Rename the columns to remove the space and special characters
columns_dict = {
'Court/Estate Name': 'Court_Estate_Name', 
'Transaction Price (HK$)': 'Transaction_Price',
'Saleable Floor Area (sq ft/sq m) [Note1]': 'Saleable_Floor_Area',
'Floor [Note2]': 'Floor_Level',
'Transaction Price per sq ft/sq m (Saleable Floor Area) (HK$)': 'Transaction_Price_sq_ft_sq_m',
'Discount Rate (%)': 'Discount_Rate',
'Agency (A)/Self-negotiation (S)': 'agency_self_negotiation'
}

overall_transaction_df = overall_transaction_df.rename(columns = columns_dict)

# separate columns into sq ft and sq m
overall_transaction_df[['Saleable_Floor_Area_sq_ft', 'Saleable_Floor_Area_sq_m']] = overall_transaction_df['Saleable_Floor_Area'].str.split('/', expand = True)
overall_transaction_df[['Transaction_Price_sq_ft', 'Transaction_Price_sq_m']] = overall_transaction_df['Transaction_Price_sq_ft_sq_m'].str.split('/', expand = True)

# drop the original columns and unnecessary columns
overall_transaction_df = overall_transaction_df.drop(columns = ['Saleable_Floor_Area', 'Transaction_Price_sq_ft_sq_m', 'Plan'])

# Rearrange the columns order
new_order = ['Location','Court_Estate_Name','Transaction_Price','Saleable_Floor_Area_sq_ft','Saleable_Floor_Area_sq_m','Transaction_Price_sq_ft','Transaction_Price_sq_m','Floor_Level','Discount_Rate','agency_self_negotiation','District','Year' ,'Month']
overall_transaction_df = overall_transaction_df[new_order]

In [214]:
overall_transaction_df.to_csv('HOS_transaction_records_2020_2025.csv', index = False, encoding = 'utf-8-sig')

In [212]:
overall_transaction_df.head()

,Location,Court_Estate_Name,Transaction_Price,Saleable_Floor_Area_sq_ft,Saleable_Floor_Area_sq_m,Transaction_Price_sq_ft,Transaction_Price_sq_m,Floor_Level,Discount_Rate,agency_self_negotiation,District,Year,Month
0,Chai Wan,Cheerful Garden,"5,700,000",592,55,"9,628","103,636",M,47,A,Eastern,2020,01
1,Chai Wan,Hang Tsui Court,"3,750,000",407,38,"9,216","99,206",M,46,S,Eastern,2020,01
2,Chai Wan,Harmony Garden,"4,500,000",431,40,"10,452","112,500",M,45,A,Eastern,2020,01
3,Chai Wan,Kai Tsui Court,"3,880,000",435,40,"8,922","96,040",H,47,A,Eastern,2020,01
4,Chai Wan,Lok Hin Terrace,"5,700,000",580,54,"9,825","105,751",M,48,A,Eastern,2020,01
